In [59]:
import pandas as pd
from datetime import datetime

def data_load():
    daily_data = pd.read_csv('data/daily_data.csv')
    products_data = pd.read_csv('data/products.csv')
    promo_data = pd.read_csv('data/promo.csv')
    return daily_data, products_data, promo_data

Описание данных:

- Daily_data - данные ежедневных продаж:

1. SKU - идентификатор продукта
2. Warehouse_code - идентификатор магазина
3. Date - дата
4. Sale_price - цена продажи
5. Balance - остаток
6. Qnt_out_sale - количество проданного товара
7. Products - данные о продуктах:

- SKU - идентификатор продукта
1. Net_weight - вес товара
2. Category_id - категория товара
3. RSL - срок годности
4. Promo - данные о промо-акциях:

- Id - идентификатор акции
1. Begin_date - дата начала акции
2. End_date - дата окончания акции
3. Period_type_id - тип акции (разный подход к маркетинговому продвижению товара)

In [60]:
daily_data, products_data, promo_data = data_load()

In [61]:
daily_data['date'] = pd.to_datetime(daily_data['date'])
promo_data['begin_date'] = pd.to_datetime(promo_data['begin_date'])
promo_data['end_date'] = pd.to_datetime(promo_data['end_date'])

In [62]:
daily_data = daily_data.drop_duplicates()
products_data = products_data.drop_duplicates()
promo_data = promo_data.drop_duplicates()

In [80]:
current_date = datetime(2024, 1, 7)
past_promos = promo_data[promo_data['end_date'] < current_date]
future_promos = promo_data[promo_data['begin_date'] >= current_date]

In [81]:
sales_with_rsl = daily_data.merge(products_data[['sku', 'rsl']], on='sku')
agg_sales = sales_with_rsl.groupby(['sku', 'warehouse_code', pd.Grouper(key='date', freq='D')]).agg({
    'sale_price': 'mean',
    'qnt_out_sale': 'sum',
    'balance': 'mean',
    'rsl': 'mean'
}).reset_index()

In [85]:
def calculate_similarity(future_promo, past_promos, aggregated_sales):
    future_sales = aggregated_sales[aggregated_sales['id'] == future_promo['id']]
    if future_sales.empty:
        return []

    future_sku = future_sales['sku'].values[0]
    future_warehouse = future_sales['warehouse_code'].values[0]
    
    # Filter past promotions for the same SKU and warehouse
    past_sales = aggregated_sales[(aggregated_sales['sku'] == future_sku) & 
                                  (aggregated_sales['warehouse_code'] == future_warehouse) & 
                                  (aggregated_sales['id'] != future_promo['id'])]

    if past_sales.empty:
        return []
    
    # Calculate similarity for each past promotion
    past_sales = past_sales.copy()  # Create a copy to avoid SettingWithCopyWarning
    past_sales.loc[:, 'qnt_diff'] = abs(future_sales['qnt_out_sale'].values[0] - past_sales['qnt_out_sale'])
    past_sales.loc[:, 'price_diff'] = abs(future_sales['sale_price'].values[0] - past_sales['sale_price'])
    past_sales.loc[:, 'balance_diff'] = abs(future_sales['balance'].values[0] - past_sales['balance'])
    past_sales.loc[:, 'rsl_diff'] = abs(future_sales['rsl'].values[0] - past_sales['rsl'])
    past_sales.loc[:, 'similarity'] = past_sales['qnt_diff'] + past_sales['price_diff'] + past_sales['balance_diff'] + past_sales['rsl_diff']
    
    # Get top 3 most similar promotions
    top_similar = past_sales.sort_values(by='similarity').head(3)
    return top_similar[['id', 'similarity']].values.tolist()

In [86]:
def get_top_similar_promos(current_promo, past_promos_data, daily_sales_data):
    # Aggregate sales data by 'sku', 'warehouse_code' and date range
    sales_with_rsl = daily_sales_data.merge(products_data[['sku', 'rsl']], on='sku')
    agg_sales = sales_with_rsl.groupby(['sku', 'warehouse_code', pd.Grouper(key='date', freq='D')]).agg({
        'sale_price': 'mean',
        'qnt_out_sale': 'sum',
        'balance': 'mean',
        'rsl': 'mean'
    }).reset_index()
    
    # Merge the aggregated sales data with the promotions data to get the aggregated sales for each promotion period
    promo_sales = sales_with_rsl.merge(past_promos_data, on=['sku', 'warehouse_code'])
    promo_sales = promo_sales[(promo_sales['date'] >= promo_sales['begin_date']) & (promo_sales['date'] <= promo_sales['end_date'])]
    
    aggregated_sales = promo_sales.groupby(['id', 'sku', 'warehouse_code']).agg({
        'qnt_out_sale': 'sum',
        'sale_price': 'mean',
        'balance': 'mean',
        'rsl': 'mean'
    }).reset_index()
    
    similar_promos = calculate_similarity(current_promo, past_promos_data, aggregated_sales)
    return similar_promos

In [88]:
# Example current promotion and previous promotions data
current_promo = {'id': 997, 'sku': 13350850, 'begin_date': datetime(2024, 2, 5), 'warehouse_code': 278850}
past_promos_data = promo_data

# Get top 3 similar promotions
top_similar_promos = get_top_similar_promos(current_promo, past_promos_data, daily_data)
print(top_similar_promos)

[[916.0, 38.471428571428575], [917.0, 38.471428571428575], [860.0, 43.88095238095238]]
